In [14]:
import pandas as pd 

In [15]:
df = pd.read_csv('data/data.csv', encoding='latin1')

In [16]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [18]:
df.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [19]:
df.shape

(541909, 8)

## Data Cleaning

In [22]:
# remove alphabets from the invoice column
df = df[~df['InvoiceNo'].str.contains('C', na=False)]

In [23]:
df = df.dropna(subset=['CustomerID'])

In [24]:
df = df[df['Quantity'] > 0]

In [28]:
# Keep only numeric stock codes
df = df[df['StockCode'].str.isnumeric()]

In [29]:
df['StockCode'].unique()

array(['71053', '22752', '21730', ..., '23561', '90089', '23843'],
      shape=(2785,), dtype=object)

In [ ]:
# Remove rows with no description
df = df[df['Description'].notna()]

In [31]:
# Add Total Price
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 363119 entries, 1 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    363119 non-null  object 
 1   StockCode    363119 non-null  object 
 2   Description  363119 non-null  object 
 3   Quantity     363119 non-null  int64  
 4   InvoiceDate  363119 non-null  object 
 5   UnitPrice    363119 non-null  float64
 6   CustomerID   363119 non-null  float64
 7   Country      363119 non-null  object 
 8   TotalPrice   363119 non-null  float64
dtypes: float64(3), int64(1), object(5)
memory usage: 27.7+ MB


In [ ]:
# df.isna().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
TotalPrice     0
dtype: int64

In [ ]:
#datatime format
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])